In [68]:
from typing import Tuple

import numpy as np
from rdkit.Chem import rdmolfiles

from qcio import Structure, view
from qcio.utils import _rdkit_determine_connectivity, _rdkit_mol_from_structure

In [62]:
zero = Structure.open("0.xyz")
zero_r = Structure.open("0-rotated.xyz")
four = Structure.open("4.xyz")
four_r = Structure.open("4-rotated.xyz")
four_reorder = Structure.open("4-reordered.xyz")
four_shuffled = Structure.open("4-shuffled.xyz")
ethanol = Structure.open("ethanol.xyz")

In [65]:
# view.view(zero, zero_r, four, four_r, four_reorder, four_shuffled, show_indices=False)

In [66]:
# view.view(four, show_indices=True, width=1200, height=800, same_viewer=True)

In [59]:
mol = _rdkit_mol_from_structure(four)
_rdkit_determine_connectivity(mol, four.charge)
ranks = np.array(rdmolfiles.CanonicalRankAtoms(mol, breakTies=False))
print(ranks)

[44 53 38 58 71 67 20 52  4 20 20 28 67 20 20 20 37 49 54 51 54 49  1  3
 63 63  1 31 31 31 31 31 31 39 59 39  5  5 39 59 39  5  5 48 62 43 12 12
 46 56 69 46 56 69 29 29 65 65 26 26 14 14 14 14 14 14  0 45 61  9  9  9]


!!! Warning !!! Distance between atoms 68 and 67 (0.984529 A) is suspicious.


In [56]:
np.where(ranks == 71)

(array([4]),)

In [64]:
view.view(ethanol, show_indices=True)

In [69]:
def kabsch(P: np.ndarray, Q: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Compute the optimal rotation matrix that aligns P onto Q using the Kabsch algorithm.

    Parameters:
        P (np.ndarray): An N x 3 array of coordinates (the structure to rotate).
        Q (np.ndarray): An N x 3 array of coordinates (the reference structure).

    Returns:
        R (np.ndarray): The optimal 3x3 rotation matrix.
        centroid_P (np.ndarray): The centroid of P.
        centroid_Q (np.ndarray): The centroid of Q.
    """
    # Compute centroids
    centroid_P = np.mean(P, axis=0)
    centroid_Q = np.mean(Q, axis=0)

    # Center the coordinates
    P_centered = P - centroid_P
    Q_centered = Q - centroid_Q

    # Compute covariance matrix
    H = np.dot(P_centered.T, Q_centered)
    # Singular Value Decomposition
    U, S, Vt = np.linalg.svd(H)
    # Compute rotation matrix
    R = np.dot(Vt.T, U.T)

    # Correct for reflection (ensure a proper rotation with det(R)=1)
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = np.dot(Vt.T, U.T)

    return R, centroid_P, centroid_Q


In [71]:
view.view(four, four_r, show_indices=True)

In [81]:
R, centroid_P, centroid_Q = kabsch(four_r.geometry[:3], four.geometry[:3])
rotated = Structure(symbols=four_r.symbols, geometry=np.dot(four_r.geometry - centroid_P, R.T) + centroid_Q)

In [83]:
view.view(four, rotated, same_viewer=True, show_indices=False)